# Lab 3 Advanced

## Linear Regression

### Model

Linear model: $y_i=\beta_0+\beta_1 x_{i1}+\dots+\beta_p x_{ip}+\epsilon$ for $i=1,\dots,n$

Matrix notation: $y=X\beta+\epsilon$

- $y$ - response variable
- $x_1, \dots, x_p$ - set of $p$ regressors
- $\epsilon$ - noise

Approximation: $\hat{y}=X\hat{\beta}$

### Ordinary Least Squares

Goal: Find values for $\beta$ that "best" fit the data

Question: What's the definition of "best"?

Error in predictions is the difference between the actual value and the predicted value: $y-\hat{y}$
![Image](https://i1.wp.com/statisticsbyjim.com/wp-content/uploads/2017/04/residuals.png?resize=300%2C186&ssl=1)

Squaring the difference accounts for overprediction and underprediction: $(y-\hat{y})^2$
![image](https://miro.medium.com/max/628/1*uBnjPy5o59FfkkMEJL0Nqw.jpeg)

The motivation behind OLS is minimizing the sum of squared errors: $\hat{\beta} = \underset{\beta}{\operatorname{argmin}} ||y-\hat{y}||_2^2$

OLS is BLUE.

### Assumptions of OLS

- Linearity - $E[y]=X\beta$
- Strict exogeneity - $E[\epsilon|X]=0$
- No perfect multicollinearity - Regressors can't be linearly dependent, X has full rank, $\Pr[\text{rank}(X)=p]=1$
- Independent errors
- Homoscedasticity - $E[\epsilon_i^2|X]=\sigma^2$
- No autocorrelation - $E[\epsilon_i\epsilon_j|X]=0$ for $i\neq j$

### Solving OLS

Goal: Minimize our objective $||y-\hat{y}||_2^2$

Idea: Take the derivative, set equal to 0, and solve for $\hat{\beta}$

\begin{align*}
||y-\hat{y}||_2^2 &= (y-\hat{y})^T (y-\hat{y}) \\
&= (y-X\hat{\beta})^T (y-X\hat{\beta}) \\
&= y^Ty - \hat{\beta}^TX^ty - y^TX\hat{\beta} + \hat{\beta}^TX^TX\hat{\beta} \\
&= y^Ty - 2\hat{\beta}^TX^ty + \hat{\beta}^TX^TX\hat{\beta} \\
\\
\nabla_\beta ||y-\hat{y}||_2^2 &= -2X^Ty + 2X^TX\hat{\beta} \\
\\
-2X^Ty + 2X^TX\hat{\beta} &= 0 \\
\Rightarrow X^TX\hat{\beta} &= X^Ty \\
\Rightarrow \hat{\beta} &= (X^TX)^{-1}X^Ty
\end{align*}

In [1]:
import sklearn as sk
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)

## Implementing OLS

### Generate a dataset

Generate 1000 samples of $X_1 \sim U(0, 10)$

Generate 1000 samples of $X_2 \sim U(-20, -10)$

Generate $y = 2 + 10X_1 - 5X_2 + \epsilon$ where $\epsilon \sim N(0, 1)$

In [2]:
# Insert code here
x_1 = np.random.uniform(0,10,1000)
x_2 = np.random.uniform(-20,-10,1000)
y = 2 + 10*x_1 - 5*x_2 + np.random.normal(0, 1, 1000)
print(x_1.shape)
print(x_2.shape)
print(y.shape)


(1000,)
(1000,)
(1000,)


### Solve for $\beta$

1. Use the closed form solution derived above (HINT: you need to add a column of ones)

2. Use sklearn's linear regression

Check that estimated coefficients from methods 1 and 2 are the same and match the data generating process.

Why do we need to add a column of ones for method 1? What happens if we don't add any noise ($\epsilon$) to our data?

In [3]:
# Insert code here
#all_data = np.hstack((my_data, new_col))
ones_col =  np.ones(1000);
print(x_1.shape)
print(ones_col.shape)
# add all x1, x2 and a ones column into matrix
X = np.column_stack((x_1, x_2))
# print('stacked shape without ones', X.shape)
# print('x', X)
X_matrix = np.column_stack((ones_col, x_1, x_2))
# print('stacked shape with ones', X_matrix.shape)
# print('X_matrix shape', X_matrix.shape )
# print(X_matrix)
# print('X_matrix.T shape', X_matrix.T.shape )
# print(X_matrix.T)
# print('X_matrix.T.dot(X_matrix)', X_matrix.T.dot(X_matrix))
# print('(X_matrix.T.dot(X_matrix)).dot(X_matrix.T)', (X_matrix.T.dot(X_matrix)).dot(X_matrix.T))

# from solution
print('\nWith ones column:')
comput =(X_matrix.T.dot(X_matrix)).dot(X_matrix.T).dot(y)
print('Before invert', (X_matrix.T.dot(X_matrix)).dot(X_matrix.T).dot(y))
print('After invert', np.linalg.inv(X_matrix.T.dot(X_matrix)).dot(X_matrix.T).dot(y))
print('sk linear regression  ', sk.linear_model.LinearRegression().fit(X, y).coef_)

print('\nWithout ones column:')
print('After invert (without noise)', np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y))
print('sk linear regression  ', sk.linear_model.LinearRegression().fit(X, y).coef_)


(1000,)
(1000,)

With ones column:
Before invert [ 3.22237853e+10  1.65691534e+11 -4.96822341e+11]
After invert [ 1.81762706  9.99255568 -5.01306829]
sk linear regression   [ 9.99255568 -5.01306829]

Without ones column:
After invert (without noise) [10.03025659 -5.11836983]
sk linear regression   [ 9.99255568 -5.01306829]


In [4]:
# Insert answers here
print('The ones columns is needed to create an offset for the coefficient calculation. ')

The ones columns is needed to create an offset for the coefficient calculation. 


## Regularization

Regularization helps prevent overfitting and increases the generalizability of your model. By adding regularization, we are reducing variance at the expense of bias in our model. The two most common forms of regularization for linear regression are ridge regression and LASSO. These two regularization methods add a L2 or L1 penalty term to the objective, respectively.

Ridge regression: $||y-\hat{y}||_2^2 + \lambda||\beta||_2^2$

LASSO: $||y-\hat{y}||_2^2 + \lambda||\beta||_1$

Ridge regression is able to shrink all coefficients towards 0 while LASSO can set some coefficients towards 0. Because of this, LASSO is also able to perform feature selection. The last section of this blog post explains this concept visually. https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b

In order to effectively use regularization, all the regressors ($X$) must be standardized so coefficients can be compared with each other and penalized accordingly. For both of these methods, the regularization parameters needs to be tuned.

Why does standardization change the coefficients but not the $R^2$ score of OLS?

Why does standarization change the coefficients and the $R^2$ score for LASSO?

Derive the closed form solution to ridge regression (HINT: you should get $\hat{\beta}=(X^TX+\lambda I)^{-1}X^Ty$)

In [5]:
# Insert answers here
print('OLS shrinks all coefficients and keeps the same predicted values,')
print('  making r2 remain constant.')
print('LASSO shrinks some coefficients, but not all,')
print('  changing the predicted values, and therefore, the r2 is changed.')

OLS shrinks all coefficients and keeps the same predicted values,
  making r2 remain constant.
LASSO shrinks some coefficients, but not all,
  changing the predicted values, and therefore, the r2 is changed.


In [6]:
# Insert derivation here

# Manual attempt based on formula
ridge = np.linalg.inv(X_matrix.T.dot(X_matrix) + 0.5).dot(X_matrix.T).dot(y)
print('ridge', ridge)

# With sklearn's Ridge
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.05)
rr.fit(X_matrix, y)
print(rr.coef_)


ridge [ 1.71763012  9.99416147 -5.01902585]
[ 0.          9.99249635 -5.0130399 ]


## Effects of collinearity

Using the dataset from the previous section, add another variable $X_3 = .5 X_1 + .5 X_2 + \epsilon$ where $\epsilon \sim N(0, 0.1)$

Generate $y = 2 + 10X_1 - 5X_2 + 7X_3 + \epsilon$ where $\epsilon \sim N(0, 1)$

Do not standardize your variables

In [12]:
# Insert code here
# Insert code here
# ones_col = np.ones(1000);
# x_1 = np.random.uniform(0,10,1000)
# x_2 = np.random.uniform(-20,-10,1000)
x_3 = 0.5*x_1 + 0.5*x_2 + np.random.normal(0,0.1,1000)
# print(x_3)
y_col = 2 + 10*x_1 - 5*x_2 + 7*x_3 + np.random.normal(0, 1, 1000)
# print(y)
X_matrix = np.column_stack((x_1, x_2, x_3))



Use OLS to estimate the coefficients. How close are they to $\beta$?

Use ridge regression with the default hyperparameters to estimate the coefficients. How close are they to $\beta$?

Use LASSO with the default hyperparameters to estimate the coefficients. How close are they to $\beta$?

Explain the behavior of these three methods.

What would happen if $X_3 = .5X_1 + .5X_2$?

In [8]:
# Insert code here
print(sk.linear_model.LinearRegression().fit(X_matrix, y_col).coef_)
print(sk.linear_model.Ridge().fit(X_matrix, y_col).coef_)
print(sk.linear_model.Lasso().fit(X_matrix, y_col).coef_)

[10.23552872 -4.7690983   6.51800259]
[10.40485227 -4.59740971  6.17625113]
[13.36937373 -1.39195893  0.        ]


In [16]:
# Insert answers here
# x_1 = np.random.uniform(0,10,1000)
# x_2 = np.random.uniform(-20,-10,1000)
x_3 = 0.5*x_1 + 0.5*x_2
# print(x_3)
y_col = 2 + 10*x_1 - 5*x_2 + 7*x_3 + np.random.normal(0, 1, 1000)
# print(y)
X_matrix = np.column_stack((x_1, x_2, x_3))
print(sk.linear_model.LinearRegression().fit(X_matrix, y_col).coef_)
print(sk.linear_model.Ridge().fit(X_matrix, y_col).coef_)
print(sk.linear_model.Lasso().fit(X_matrix, y_col).coef_)

print('OLS is badly affected by the uniformity of X3 to X1 and X2. It breaks the OLS rule "No perfect multicollinearity"')
print('Rindge is less aaffected')
print('LASSO is barely affected')



[ 8.08988892e+10  8.08988892e+10 -1.61797778e+11]
[11.49553931 -3.48289006  4.00632463]
[13.38095199 -1.36754194  0.        ]
OLS is badly affected by the uniformity of X3 to X1 and X2. It breaks the OLS rule "No perfect multicollinearity"
Rindge is less aaffected
LASSO is barely affected


### Image sources
- https://statisticsbyjim.com/glossary/ordinary-least-squares/
- https://medium.com/@saahil1292/machine-learning-102-linear-regression-ordinary-least-squares-ols-correlation-and-analysis-of-7d53751ea9f4